In [73]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch

### Creating train, test data

In [9]:
df = pd.read_csv('./data/Microsoft_Stock.csv', index_col = 0, parse_dates = True )
df.index = df.index.date

In [12]:
df.head()

,Open,High,Low,Close,Volume
2015-04-01,40.60,40.76,40.31,40.72,36865322
2015-04-02,40.66,40.74,40.12,40.29,37487476
2015-04-06,40.34,41.78,40.18,41.55,39223692
2015-04-07,41.61,41.91,41.31,41.53,28809375
2015-04-08,41.48,41.69,41.04,41.42,24753438


In [13]:
len_train = len(df) * 0.7
len_test = len(df) * 0.3

In [109]:
len_train, len_test

(1057.7, 151.1)

In [14]:
train_df = df.iloc[ :int(len_train+len_test)+1 ,:]
test_df = df.iloc[int(len_train+len_test)+1: , :]

In [74]:
scaler = MinMaxScaler()
open_train = train_df[['Open']]  
scaler.fit(open_train)

x_train = scaler.transform(open_train)
x_test = scaler.transform(test_df[['Open']])

In [26]:
np.save('./data/train.npy', x_train)
np.save('./data/test.npy', x_test)

### Predictions

In [108]:
from data import TimeSeriesDataset

x_test = np.load('./data/test.npy')
x_test_tensor = torch.Tensor(x_test)
test_dataset = TimeSeriesDataset(x_test_tensor, 25)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [91]:
x_test.shape

(302, 1)

In [106]:
model = torch.load('resnet.pt')
model.eval()  
predictions = []
actual_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:  
    
        inputs = torch.flatten(inputs, start_dim=1)

        outputs = model(inputs)

        predictions.append(outputs.cpu().numpy())
        actual_labels.append(labels.cpu().numpy())


predictions = np.concatenate(predictions, axis=0)
actual_labels = np.concatenate(actual_labels, axis=0)

In [107]:
mse = np.mean((predictions - actual_labels) ** 2)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.004646168
